In [5]:
%pip install --upgrade pandas
%pip install --upgrade nltk

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
from collections import Counter
import itertools 
import re
# import contractions

import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('treebank')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /Users/leomoore/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
df_a = pd.read_csv("fake_job_postings_A.csv")
df_b = pd.read_csv("fake_job_postings_B.csv")

df = pd.concat([df_a, df_b], ignore_index=True)

df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,Unnamed: 8,0,0.1,0.2,Full-time,Unnamed: 13,Unnamed: 14,Management Consulting,Unnamed: 16,0.3
0,1.0,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Remove weird characters from description column, convert to all lowercase

df["description"] = df["description"].str.replace(r"[^\w\s]", "", regex=True)
df["description"] = df["description"].str.lower()

In [59]:
# Set parameters for stop words and min word length, to be used later
global_stopwords = nltk.corpus.stopwords.words("english")
THRESHOLD = 3

In [61]:
# Get list of words in the fraudulent descriptions

fraudulent_description = df[df["fraudulent"] == 1]["description"]

fraud_words = [nltk.word_tokenize(s) for s in fraudulent_description if isinstance(s, str)]
fraud_words = list(itertools.chain.from_iterable(fraud_words))
fraud_words = [w for w in fraud_words if len(w)> THRESHOLD and w not in global_stopwords]
fraud_words

['icampe',
 'technician',
 'bakersfield',
 'posoprincipal',
 'duties',
 'responsibilities',
 'calibrates',
 'tests',
 'maintains',
 'troubleshoots',
 'installs',
 'power',
 'plant',
 'instrumentation',
 'control',
 'systems',
 'electrical',
 'equipmentperforms',
 'maintenance',
 'motor',
 'control',
 'centers',
 'motor',
 'operated',
 'valves',
 'generators',
 'excitation',
 'equipment',
 'motorsperforms',
 'preventive',
 'predictive',
 'corrective',
 'maintenance',
 'equipment',
 'coordinating',
 'work',
 'various',
 'team',
 'membersdesigns',
 'installs',
 'equipment',
 'andor',
 'system',
 'modificationstroubleshoots',
 'performs',
 'maintenance',
 'backup',
 'power',
 'equipment',
 'process',
 'controls',
 'programmable',
 'logic',
 'controls',
 'emission',
 'monitoring',
 'equipmentuses',
 'maintenance',
 'reporting',
 'system',
 'record',
 'time',
 'material',
 'problem',
 'identified',
 'corrected',
 'action',
 'required',
 'provides',
 'complete',
 'history',
 'maintenance',
 '

In [64]:
# Get most common words in fraudulent descriptions

fraud_counts = Counter(fraud_words)
fraud_counts.most_common(20)
fraud_freq = pd.DataFrame(fraud_counts.most_common(), columns=["Fraud_Term", "Frequency"])
fraud_freq.head(10)

,Fraud_Term,Frequency
0,work,495
1,project,296
2,management,295
3,position,263
4,team,254
5,business,252
6,company,238
7,support,226
8,customer,222
9,solutions,220


In [66]:
# Get list of words in the legitimate descriptions

legitimate_description = df[df["fraudulent"] == 0]["description"]

legit_words = [nltk.word_tokenize(s) for s in legitimate_description if isinstance(s, str)]
legit_words = list(itertools.chain.from_iterable(legit_words))
legit_words = [w for w in legit_words if len(w)> THRESHOLD and w not in global_stopwords]
legit_words

['food52',
 'fastgrowing',
 'james',
 'beard',
 'awardwinning',
 'online',
 'food',
 'community',
 'crowdsourced',
 'curated',
 'recipe',
 'currently',
 'interviewing',
 'full',
 'parttime',
 'unpaid',
 'interns',
 'work',
 'small',
 'team',
 'editors',
 'executives',
 'developers',
 'york',
 'city',
 'headquartersreproducing',
 'andor',
 'repackaging',
 'existing',
 'food52',
 'content',
 'number',
 'partner',
 'sites',
 'huffington',
 'post',
 'yahoo',
 'buzzfeed',
 'various',
 'content',
 'management',
 'systemsresearching',
 'blogs',
 'websites',
 'provisions',
 'food52',
 'affiliate',
 'programassisting',
 'daytoday',
 'affiliate',
 'program',
 'support',
 'screening',
 'affiliates',
 'assisting',
 'affiliate',
 'inquiriessupporting',
 'events',
 'neededhelping',
 'office',
 'administrative',
 'work',
 'filing',
 'mailing',
 'preparing',
 'meetingsworking',
 'developers',
 'document',
 'bugs',
 'suggest',
 'improvements',
 'sitesupporting',
 'marketing',
 'executive',
 'staff',
 '

In [67]:
# Get most common words in legitimate descriptions

legit_counts = Counter(legit_words)
legit_counts.most_common(20)
legit_freq = pd.DataFrame(legit_counts.most_common(), columns=["Legit_Term", "Frequency"])
legit_freq.head(10)

,Legit_Term,Frequency
0,team,8377
1,work,6289
2,business,5134
3,experience,4486
4,sales,4377
5,company,4353
6,customer,4350
7,looking,4085
8,development,3868
9,management,3435
